In [1]:
from __future__ import print_function

%matplotlib inline

import sys
import tensorflow as tf
from collections import OrderedDict

import numpy as np
import scipy.misc
from input import *
from model import *
from train import *
import matplotlib.pyplot as plt

1.3.0


In [2]:
tf.reset_default_graph()

In [3]:
iterator, filename = get_train_inputs(batch_size=100,repeat=False,num_classes=2)

In [ ]:
i,l = iterator.get_next()

In [ ]:
# with tf.Session() as sess:
#     sess.run(iterator.initializer, feed_dict={filename: ['data/kitti_segmentation.tfrecord']})
#     next_element = iterator.get_next()
#     while True:
#         image, label = sess.run(next_element)
#         plt.imshow(image)
#         plt.show()
#     #plt.imshow(label[:,:,0], cmap='jet', alpha=0.5)

In [ ]:
# record_iterator = tf.python_io.tf_record_iterator(path='data/kitti_segmentation.tfrecord')
# string_record = next(record_iterator)
# example = tf.train.Example()
# example.ParseFromString(string_record)

# train

In [4]:
encoder = SlimModelEncoder(name="vgg_16",num_classes=2,is_training=True)

In [5]:
image, label = iterator.get_next()
init_fn, end_points = encoder.build(image)

INFO:tensorflow:Scale of 0 disables regularizer.


In [6]:
end_points.keys()

odict_keys(['vgg_16/conv1/conv1_1', 'vgg_16/conv1/conv1_2', 'vgg_16/pool1', 'vgg_16/conv2/conv2_1', 'vgg_16/conv2/conv2_2', 'vgg_16/pool2', 'vgg_16/conv3/conv3_1', 'vgg_16/conv3/conv3_2', 'vgg_16/conv3/conv3_3', 'vgg_16/pool3', 'vgg_16/conv4/conv4_1', 'vgg_16/conv4/conv4_2', 'vgg_16/conv4/conv4_3', 'vgg_16/pool4', 'vgg_16/conv5/conv5_1', 'vgg_16/conv5/conv5_2', 'vgg_16/conv5/conv5_3', 'vgg_16/pool5', 'vgg_16/fc6', 'vgg_16/fc7', 'vgg_16/fc8'])

In [7]:
end_points['vgg_16/conv5/conv5_3'], end_points['vgg_16/conv4/conv4_3'], end_points['vgg_16/conv3/conv3_3']

(<tf.Tensor 'vgg_16/conv5/conv5_3/Relu:0' shape=(1, 14, 14, 512) dtype=float32>,
 <tf.Tensor 'vgg_16/conv4/conv4_3/Relu:0' shape=(1, 28, 28, 512) dtype=float32>,
 <tf.Tensor 'vgg_16/conv3/conv3_3/Relu:0' shape=(1, 56, 56, 256) dtype=float32>)

In [8]:
decoder = FCNDecoder(end_points, nb_classes=2, scope='fcn')

In [9]:
tensors_to_connect = OrderedDict()
tensors_to_connect['vgg_16/conv5/conv5_3'] = (2,2)
tensors_to_connect['vgg_16/conv4/conv4_3'] = (2,2)
tensors_to_connect['vgg_16/conv3/conv3_3'] = (4,4)

In [10]:
net = decoder.build(tensors_to_connect)

In [11]:
optimizer = tf.train.AdamOptimizer
trainer = Trainer(nb_classes=2, learning_rate=1e-4, optimizer=optimizer)
trainer.build(net, label)

In [12]:
input_tensor = tf.get_default_graph().get_tensor_by_name('training_data/input:0')

In [14]:
trainer.train(iterator,['data/kitti_segmentation.tfrecord'])

INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path /tmp/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, logits and labels must be same size: logits_size=[50176,2] labels_size=[465750,2]
	 [[Node: softmax_cross_entropy_loss/xentropy = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](softmax_cross_entropy_loss/Reshape, softmax_cross_entropy_loss/Reshape_1)]]

Caused by op 'softmax_cross_entropy_loss/xentropy', defined at:
  File "/Users/deniz/anaconda2/envs/py3tf/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/deniz/anaconda2/envs/py3tf/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/deniz/anaconda2/envs/py3tf/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
 

InvalidArgumentError: logits and labels must be same size: logits_size=[50176,2] labels_size=[465750,2]
	 [[Node: softmax_cross_entropy_loss/xentropy = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](softmax_cross_entropy_loss/Reshape, softmax_cross_entropy_loss/Reshape_1)]]

Caused by op 'softmax_cross_entropy_loss/xentropy', defined at:
  File "/Users/deniz/anaconda2/envs/py3tf/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/deniz/anaconda2/envs/py3tf/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/deniz/anaconda2/envs/py3tf/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/deniz/anaconda2/envs/py3tf/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/deniz/anaconda2/envs/py3tf/lib/python3.6/site-packages/ipykernel